In [1]:
import os
from globals import TRAINING_DIR,MODELS_DIR, DATA_DIR, id2label,label2id
import mlflow
from datasets import load_dataset
import requests
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer)
from pprint import PrettyPrinter
import numpy as np
import evaluate
from metrics.evaluar_metricas import evaluar_metricas_NER
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score

Definición de variables globales, parámetros de entrenamiento y MLflow

In [2]:
# VARIABLES GLOBALES
train_max = None # Número máximo de elementos para entrenamiento (para pruebas) None para ir en serio
training_output_dir = os.path.join(TRAINING_DIR,"NER")
# Defino una serie de variables que registraré en los entrenamientos de MLflow
ml_params = {
    'num_epochs': 5,
    'lr' : 1e-5,
    'eval_steps' : 0.05, 
    'eval_batch_size' : 32,
    'label_all_tokens': True,
    'model_name': os.path.join(MODELS_DIR,'PlanTL-GOB-ES','roberta-base-bne-capitel-ner-plus')
}
num_epochs = lr = eval_steps = save_steps = eval_batch_size = label_all_tokens = model_name = 0
for key, value in ml_params.items():
    assert not globals()[key] is None, f'La variable global {key} debe estar definida'    
    globals()[key] = value

pp = PrettyPrinter(width=150)

Carga del conjunto de datos 

In [3]:
main_dataset = load_dataset(os.path.join(DATA_DIR,'Escrituras'), 'NER',trust_remote_code=True)
train_dataset = main_dataset['train']
val_dataset = main_dataset['validation']
if train_max:
    train_dataset = train_dataset.select(range(train_max))
    val_dataset = val_dataset.select(range(train_max))
del main_dataset

Comprobación de que el servidor MLflow está funcionando para las pruebas

In [4]:
SERVIDOR_MLFLOW = 'http://localhost:5000'
# Debo comprobar si está ejecutando el servidor MLflow, en otro caso se demora la ejecución y acaba dando un error
def mlflow_en_ejecucion(url):
    try:
        response = requests.get(url)        
        # Si el servidor está en ejecución, deberíamos recibir un código de estado HTTP 200
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        # Si no se puede establecer una conexión, asumimos que el servidor no está en ejecución
        return False
    
assert mlflow_en_ejecucion(SERVIDOR_MLFLOW), f"El servidor MLflow ({SERVIDOR_MLFLOW}) no está en ejecución. Lance 'mlflow ui' desde el terminal."

### Iniciamos el entrenamiento

In [ ]:
# Servidor de seguimiento
mlflow.set_tracking_uri(SERVIDOR_MLFLOW)
mlflow.autolog()
mlflow.set_experiment("ENTRENAMIENTO Named Entity Recognition")

In [ ]:
model  = AutoModelForTokenClassification.from_pretrained(model_name, 
                                                         num_labels=9, ignore_mismatched_sizes=True,
                                                         id2label=id2label,label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
pp.pprint(train_dataset.info)

In [ ]:
def f_preproceso(examples):
    """Función para generar los input_ids, atention_mask y otras características para el entrenamiento.
        Realinea los 'ner_tags' que"""        
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True, truncation=True, padding=True)
    
    ner_tags_ids = []
    for i, tags in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # indica de qué palabra viene cada token
        previous_word_idx = None
        tag_ids = []
        for word_idx in word_ids:  
            # Tokens especiales van a -100 para ser ignorados por la función de pérdida.
            if word_idx is None:
                tag_ids.append(-100)
            elif word_idx != previous_word_idx:  # Sólo se etiqueta la primera aparición de cada palabra
                tag_ids.append(tags[word_idx])
            else:
                tag_ids.append(tags[word_idx] if ml_params['label_all_tokens'] else -100)
                
            previous_word_idx = word_idx
        ner_tags_ids.append(tag_ids)

    tokenized_inputs["labels"] = ner_tags_ids
    return tokenized_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(f_preproceso, batched=True)
tokenized_val_dataset = val_dataset.map(f_preproceso,batched=True)
del train_dataset
del val_dataset

In [ ]:
pp.pprint(tokenized_train_dataset.info)

In [ ]:
# Cargo la lista de etiquetas definidas en el dataset
lista_etiquetas = tokenized_train_dataset.features['ner_tags'].feature.names
# Y Cargo la métrica
# seqeval = evaluate.load("seqeval")

# Función de evaluación 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    predictions_list = [
        [lista_etiquetas[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [lista_etiquetas[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
    ]

    # results = seqeval.compute(predictions=predictions_list, references=true_labels)
    # return {
    #     "precision": results["overall_precision"],
    #     "recall": results["overall_recall"],
    #     "f1_score": results["overall_f1"],
    #     "accuracy": results["overall_accuracy"],
    # }
    return {
        "precision": precision_score(true_labels, predictions_list),
        "recall":   recall_score(true_labels, predictions_list),
        "f1_score": f1_score(true_labels, predictions_list),
        "accuracy": accuracy_score(true_labels, predictions_list)
    }

Definición de los parámetros y el trainer

In [ ]:
training_args = TrainingArguments(
    output_dir=training_output_dir,
    overwrite_output_dir=True,
    do_train=True,
    learning_rate=lr,
    per_device_eval_batch_size=eval_batch_size,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=eval_steps,    
    load_best_model_at_end=True,
    metric_for_best_model='f1_score'    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,    
    compute_metrics=compute_metrics
)

In [ ]:
with mlflow.start_run(run_name=f"{'Prueba con ' + str(train_max) if train_max else 'Entrenamiento'}"):
    mlflow.autolog()
    trainer.train()
    for param_name, param_value in ml_params.items():
        mlflow.log_param(param_name, param_value)    
    

In [ ]:
# Guardar el modelo
from datetime import datetime
n_epochs = trainer.args.num_train_epochs
g_steps = trainer.state.global_step
fecha_hora = datetime.now().strftime("%Y%m%d-%H%M")
ruta_modelo_ajustado = os.path.join(MODELS_DIR,f"{fecha_hora}_escrituras_NER_{n_epochs}-epoch_{g_steps}-steps")
trainer.save_model(ruta_modelo_ajustado)
tokenizer.save_pretrained(ruta_modelo_ajustado)

Pruebas de resultados

In [ ]:
from transformers import pipeline
import torch

id_test = 3
model.eval()    
test_dataset = load_dataset(os.path.join(DATA_DIR,'Escrituras'), 'NER',trust_remote_code=True, split="test")

# # Método 3
consulta =  pipeline("ner", model=model, tokenizer=tokenizer, 
                device=0 if torch.cuda.is_available() else None, batch_size=32)        
predicciones = consulta(" ".join(test_dataset['tokens'][id_test]))
predicciones.sort(key=lambda x: x['index'])

texto = " ".join(test_dataset['tokens'][id_test])
print(texto,'\n')
last_label = None
palabra = None
for p in predicciones:    
    if p['entity'].startswith('B'):
    # if last_label != p['entity']:
        if palabra != None:            
            print(last_label[2:], '\t'+palabra)
        palabra = texto[p['start']:p['end']]
    else:
        palabra += " " + texto[p['start']:p['end']]
    last_label = p['entity']
print(last_label[2:], '\t'+palabra, '\n')
for p in predicciones:
    print(p['entity'] ,p['word'], texto[p['start']:p['end']])

## Validación final con el conjunto de test

In [6]:
mlflow.set_tracking_uri(SERVIDOR_MLFLOW)
mlflow.autolog()
mlflow.set_experiment("ENTRENAMIENTO Named Entity Recognition")

try:
    print(ruta_modelo_ajustado)
except:
    ruta_modelo_ajustado = os.path.join(MODELS_DIR,"20240218-0307_escrituras_NER_5-epoch_3765-steps")
    
test_dataset = load_dataset(os.path.join(DATA_DIR,'Escrituras'), 'NER',trust_remote_code=True, split="test")
with mlflow.start_run(run_name="Prueba de evaluación",description="Predicciones con el modelo ajustado, usando dataset de test"):
    evaluar_metricas_NER(ruta_modelo_ajustado,test_dataset)

2024/02/18 04:29:45 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/02/18 04:29:45 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.


..\Models\20240218-0307_escrituras_NER_5-epoch_3765-steps
..\Models\20240218-0307_escrituras_NER_5-epoch_3765-steps
	f1: 0.9558318531675548
